In [1]:
import numpy as np
import pandas as pd
import os
from importlib import reload
import find_cpt
from rgf.sklearn import RGFClassifier
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.svm import SVC
from sklearn.model_selection import KFold, cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score, recall_score, f1_score
data_dir = "./data"
summary = pd.read_csv('./preprocess/summarize.csv')

In [19]:
tmp = summary.loc[0].iloc[1:].sort_values(ascending=False)
print(tmp[tmp > 0.01].index)

Index(['smart_242_raw', 'smart_7_raw', 'smart_193_raw', 'smart_9_raw',
       'smart_240_raw', 'smart_190_normalized', 'smart_190_raw',
       'smart_194_raw', 'smart_194_normalized', 'smart_9_normalized',
       'smart_198_raw', 'smart_197_raw', 'smart_241_raw', 'smart_7_normalized',
       'smart_187_normalized', 'smart_187_raw', 'smart_4_raw', 'smart_12_raw',
       'smart_1_normalized', 'smart_193_normalized', 'smart_1_raw',
       'smart_5_raw', 'smart_3_normalized', 'smart_183_raw',
       'smart_183_normalized', 'smart_198_normalized', 'smart_197_normalized',
       'smart_5_normalized', 'smart_192_raw', 'smart_189_raw',
       'smart_189_normalized', 'smart_184_normalized', 'smart_184_raw'],
      dtype='object')
